In [1]:
import os
import sys 
from datetime import datetime

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt 
import seaborn as sns
# import econtools.metrics as mt
from tqdm import trange, notebook

import statsmodels.api as sm

import re

pd.set_option('display.max_columns', None)


In [3]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)
from config import directory_path
os.chdir(directory_path)
os.getcwd()


'C:\\Users\\nadav\\Dropbox\\uchicago_fourth\\uncertaintyInequality'

# Load Income Data

In [4]:
incomeData = pd.read_feather('proc_data\HH_Income_Panel_long.feather')
incomeData


,HH_ID,STATE,HR,DISTRICT,REGION_TYPE,STRATUM,PSU_ID,MONTH_SLOT,MONTH,RESPONSE_STATUS,REASON_FOR_NON_RESPONSE,HH_WEIGHT_MS,HH_WEIGHT_FOR_COUNTRY_MS,HH_WEIGHT_FOR_STATE_MS,HH_NON_RESPONSE_MS,HH_NON_RESPONSE_FOR_COUNTRY_MS,HH_NON_RESPONSE_FOR_STATE_MS,AGE_GROUP,OCCUPATION_GROUP,EDUCATION_GROUP,GENDER_GROUP,SIZE_GROUP,TOTAL_INCOME,INCOME_OF_ALL_MEMBERS_FROM_ALL_SOURCES,INCOME_OF_ALL_MEMBERS_FROM_WAGES,INCOME_OF_ALL_MEMBERS_FROM_PENSION,INCOME_OF_ALL_MEMBERS_FROM_DIVIDEND,INCOME_OF_ALL_MEMBERS_FROM_INTEREST,INCOME_OF_HOUSEHOLD_FROM_ALL_SOURCES,INCOME_OF_HOUSEHOLD_FROM_RENT,INCOME_OF_HOUSEHOLD_FROM_SELF_PRODUCTION,INCOME_OF_HOUSEHOLD_FROM_PRIVATE_TRANSFERS,INCOME_OF_HOUSEHOLD_FROM_GOVERNMENT_TRANSFERS,INCOME_OF_HOUSEHOLD_FROM_BUSINESS_PROFIT,INCOME_OF_HOUSEHOLD_FROM_SALE_OF_ASSET,INCOME_OF_HOUSEHOLD_FROM_GAMBLING,FAMILY_SHIFTED,INCOME_OF_ALL_MEMBERS_FROM_FD_PF_INSURANCE
0,77051739,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Youngsters - dominant,White-collar Clerical Employees,All Graduates household,Female Majority,8-10 Members,19000,19000,19000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
1,61287374,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Balanced households with no Seniors,White-collar Professional Employees,All Graduates household,Male Dominated,4 Members,20000,20000,20000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
2,34049553,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Grown-up - dominant,Self-employed Entrepreneurs,All Matriculates household,Male Majority,3 Members,5000,5000,5000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
3,92526123,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Others households of Grown-ups,Wage Labourers,Matriculates minority household,Male Majority,8-10 Members,25000,25000,25000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
4,61508707,Jammu & Kashmir,HR 1,Bandipore,RURAL,HR 1_RURAL_R,PSU_ID_7959,May 2015,Jan 2015,Accepted,No Failure,2707.736909,2707.736909,2707.736909,1.103118,1.103118,1.103118,Others households of Grown-ups,Small Traders/Hawkers,Households of some literates,Balanced,4 Members,9000,9000,9000,0,0,0,0,0,0,0,0,0,0,0,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9720289,18693357,Meghalaya,HR 110,East Khasi Hills,RURAL,HR 110_RURAL_R,PSU_ID_12896,Mar 2020,Nov 2019,Accepted,No Failure,780.358644,780.358644,780.358644,1.169435,1.169435,1.169435,Youngsters - dominant,Small/Marginal Farmers,Households of all literates,Male Majority,7 Members,43725,42750,42750,0,0,0,975,0,975,0,0,0,0,0,N,0.0
9720290,82768393,Meghalaya,HR 110,East Khasi Hills,RURAL,HR 110_RURAL_R,PSU_ID_12896,Mar 2020,Nov 2019,Accepted,No Failure,780.358644,780.358644,780.358644,1.169435,1.169435,1.169435,Youngsters - dominant,Non-industrial Technical Employees,Households of all literates,Balanced,4 Members,18332,17632,17632,0,0,0,700,0,700,0,0,0,0,0,N,0.0
9720291,18077972,Meghalaya,HR 110,East Khasi Hills,RURAL,HR 110_RURAL_R,PSU_ID_12896,Mar 2020,Nov 2019,Accepted,No Failure,780.358644,780.358644,780.358644,1.169435,1.169435,1.169435,Other households of the Young,Small/Marginal Farmers,Matriculates majority household,Balanced,4 Members,19104,18520,18520,0,0,0,584,0,584,0,0,0,0,0,N,0.0
9720292,35087702,Meghalaya,HR 110,East Khasi Hills,RURAL,HR 110_RURAL_R,PSU_ID_12896,Mar 2020,Nov 2019,Accepted,No Failure,780.358644,780.358644,780.358644,1.169435,1.169435,1.169435,Other households of the Young,Small/Marginal Farmers,Households of all literates,Male Majority,8-10 Members,43000,41800,41800,0,0,0,1200,0,1200,0,0,0,0,0

In [7]:
consumptionData = pd.read_feather('proc_data\HH_consumption_long.feather')
consumptionData



,HH_ID,TOTAL_EXPENDITURE,MONTHLY_EXPENSE_ON_FOOD,MONTHLY_EXPENSE_ON_INTOXICANTS,MONTHLY_EXPENSE_ON_CLOTHING_AND_FOOTWEAR,MONTHLY_EXPENSE_ON_COSMETIC_AND_TOILETRIES,MONTHLY_EXPENSE_ON_APPLIANCES,MONTHLY_EXPENSE_ON_RESTAURANTS,MONTHLY_EXPENSE_ON_RECREATION,MONTHLY_EXPENSE_ON_BILLS_AND_RENT,MONTHLY_EXPENSE_ON_POWER_AND_FUEL,MONTHLY_EXPENSE_ON_TRANSPORT,MONTHLY_EXPENSE_ON_COMMUNICATION_AND_INFO,MONTHLY_EXPENSE_ON_EDUCATION,MONTHLY_EXPENSE_ON_HEALTH,MONTHLY_EXPENSE_ON_ALL_EMIS,MONTHLY_EXPENSE_ON_MISCELLANEOUS,REASON_FOR_NON_RESPONSE,FAMILY_SHIFTED,MONTH_SLOT,MONTH
0,77051739,16820,7460,400,0,570,0,0,0,100,3300,600,900,800,180,0,2510,No Failure,N,May 2015,Jan 2015
1,61287374,10253,5900,300,1000,483,0,0,0,60,1100,500,500,0,160,0,250,No Failure,N,May 2015,Jan 2015
2,34049553,8967,5370,180,0,207,600,0,0,30,700,420,450,0,600,0,410,No Failure,N,May 2015,Jan 2015
3,92526123,15265,6380,600,400,435,0,500,0,50,3100,670,750,600,80,0,1700,No Failure,N,May 2015,Jan 2015
4,61508707,9155,5090,400,0,355,0,0,0,50,800,830,500,0,140,0,990,No Failure,N,May 2015,Jan 2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9720289,18693357,12768,7122,900,650,886,0,150,170,0,525,350,300,675,205,0,835,No Failure,N,Mar 2020,Nov 2019
9720290,82768393,12253,7413,864,250,1068,0,180,0,0,654,360,310,400,239,0,515,No Failure,N,Mar 2020,Nov 2019
9720291,18077972,11506,7801,680,0,550,0,200,145,0,740,420,400,45,245,0,280,No Failure,N,Mar 2020,Nov 2019
9720292,35087702,16141,9880,1205,0,1061,0,175,175,0,600,450,700,895,210,0,790,No Failure,N,Mar 2020,Nov 2019


In [5]:
HHInfo = pd.read_feather('proc_data\HH_panel_long.feather')
HHInfo = HHInfo.sort_values(['HH_ID','MONTH_SLOT','AGE_YRS'])
HHInfo = HHInfo.groupby(['HH_ID','MONTH_SLOT']).first().reset_index() #There are two cases in which we have two HOH - so we keep the older of the two
HHInfo

,HH_ID,MONTH_SLOT,RELIGION,CASTE,CASTE_CATEGORY,LITERACY,EDUCATION,DISCIPLINE,NATURE_OF_OCCUPATION,INDUSTRY_OF_OCCUPATION,AGE_YRS,AGE_MTHS,GENDER,under18IND,not_healthy,Sons,Daughters,Parents,livingWSpouse
0,10000111,Apr 2015,Muslim,Mughal (Khan),Upper Caste,N,No Education,Not Applicable,Home Maker,Not Applicable,62,0,F,13,0,1,0,0,0
1,10000111,Jul 2018,Muslim,Mughal (Khan),Upper Caste,Y,5th Std. Pass,Not Applicable,Home Maker,Not Applicable,60,0,F,13,1,1,0,0,0
2,10000111,Jul 2019,Muslim,Mughal (Khan),Upper Caste,Y,5th Std. Pass,Not Applicable,Home Maker,Not Applicable,67,0,F,13,1,1,0,0,0
3,10000111,Mar 2017,Muslim,Mughal (Khan),Upper Caste,Y,5th Std. Pass,Not Applicable,Retired/Aged,Not Applicable,65,0,F,13,0,1,0,0,0
4,10000111,Mar 2018,Muslim,Mughal (Khan),Upper Caste,Y,5th Std. Pass,Not Applicable,Home Maker,Not Applicable,60,0,F,13,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2276726,99998893,Nov 2019,Hindu,Other OBC,OBC,Y,6th Std. Pass,Not Applicable,Wage Labourer,Real Estate & Construction,45,0,M,1,0,2,1,0,1
2276727,99998894,Jul 2018,Hindu,Brahmin,Upper Caste,Y,Graduate,Arts,White Collar Clerical Employees,Public Administrative Services,35,0,M,1,0,1,0,0,1
2276728,99998894,Jul 2020,Hindu,Brahmin,Upper Caste,Y,Graduate,Arts,White Collar Clerical Employees,Defence Services,34,0,M,1,0,1,0,0,1
2276729,99998894,Mar 2019,Hindu,Brahmin,Upper Caste,Y,Graduate,Arts,White-Collar Professional Employees and Other ...,Public Administrative Services,34,0,M,1,0,1,0,0,1


In [6]:
# Calculate Returns
incomeData['TotalReturns'] = incomeData['INCOME_OF_ALL_MEMBERS_FROM_DIVIDEND']+ \
incomeData['INCOME_OF_ALL_MEMBERS_FROM_INTEREST']+ \
incomeData['INCOME_OF_HOUSEHOLD_FROM_RENT']+ \
incomeData['INCOME_OF_HOUSEHOLD_FROM_BUSINESS_PROFIT']

# Calculate Total Income
incomeData['totalIncome'] = incomeData['INCOME_OF_ALL_MEMBERS_FROM_WAGES']+ \
incomeData['INCOME_OF_ALL_MEMBERS_FROM_PENSION']+ incomeData['INCOME_OF_HOUSEHOLD_FROM_SELF_PRODUCTION']+ \
incomeData['INCOME_OF_HOUSEHOLD_FROM_GOVERNMENT_TRANSFERS'] + incomeData['INCOME_OF_HOUSEHOLD_FROM_GAMBLING']

incomeData = incomeData[['HH_ID','MONTH','MONTH_SLOT','totalIncome','TotalReturns','REASON_FOR_NON_RESPONSE','FAMILY_SHIFTED']]

In [ ]:
consumptionData = consumptionData[['HH_ID','MONTH','TOTAL_EXPENDITURE','REASON_FOR_NON_RESPONSE', 'FAMILY_SHIFTED']]

In [ ]:
incomeConsumptionData = incomeData.merge(consumptionData,on=['HH_ID','MONTH'],how='inner',validate='1:1')

In [ ]:
incomeConsumptionData = incomeConsumptionData.merge(HHInfo,on=['HH_ID','MONTH_SLOT'],how='left',validate='m:1') #It seems that sometimes some observations are missing in the HH_ID. But we still have consumption data and so on. So I don't ommit it

In [ ]:
incomeConsumptionData.to_stata('proc_data/interestRate_longData.dta')